In [1]:
import numpy as np
from collections import Counter
import math
from numba import jit
from fractions import Fraction

In [2]:
NUM_DICE = 5

In [3]:
def roll_dices(keepers=[]):
    return keepers + [np.random.randint(1, 7) for _ in range(NUM_DICE - len(keepers))]

In [4]:
def select(n, m):
    C = math.factorial(n)/(math.factorial(m)*math.factorial(n-m)) 
    return C

In [5]:
# Calculates the chance of going from one amount of a number to another in one dice roll
def prob1(fro, to):
    n = NUM_DICE - fro 
    m = to - fro
    C = select(n, m)
    prob = C*math.pow(1.0/6.0,(m))*math.pow(5.0/6.0,(n-m)) 
    return prob

In [6]:
# Calculates the chance of going from one amount of a number to another in two dice rolls
def prob2(fro, to):
    prob = 0
    for i in range(fro,to+1):
        prob += prob1(fro,i)*prob1(i,to) 
    return prob

In [7]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [8]:
def cmb_keepers(roll):
    return list(set(list(powerset(roll))))

In [9]:
def upper_bonus(score_card):
    total = 0
    for cat in ['aces', 'twos', 'threes', 'fours', 'fives', 'sixes']:
        total += score_card[cat] if score_card[cat] != -1 else 0
    
    return 35 if total >= 63 else 0

In [10]:
@jit()
def score_3ofkind(roll):
    for e in roll:
        if roll.count(e) >= 3:
            return sum(roll)
    return 0

In [11]:
@jit()
def score_4ofkind(roll):
    for e in roll:
        if roll.count(e) >= 4:
            return sum(roll)
    return 0

In [12]:
@jit()
def score_fullhouse(roll):
    if ((roll.count(list(set(roll))[0]) == 3 and roll.count(list(set(roll))[1]) == 2)
       or (roll.count(list(set(roll))[0]) == 2 and roll.count(list(set(roll))[1]) == 3)):
        return 25
    return 0

In [13]:
def score_smsrtaight(roll):
    roll = list(set(roll))
    roll.sort()
    s = ''.join(str(x) for x in roll)
    for st in ['1234','2345','3456']:
        if st in s:
            return 30
    return 0

In [14]:
def score_lstraight(roll):
    roll = list(set(roll))
    roll.sort()
    s = ''.join(str(x) for x in roll)
    if s == '12345' or s == '23456':
        return 40
    return 0

In [15]:
@jit()
def score_yahtzee(roll):
    return 50 if roll.count(roll[0]) == 5 else 0

In [16]:
# gibt alle einzigartigen Kombinationen von amt verschiedenen Würfeln zurück
def cmb_rolls(amt):
    def _rolls(amt, min_v):
        if amt == 0:
            return [[]]
        
        output = []
        
        for i in range(min_v, 7):
            next_roll = _rolls(amt-1, i)
            for j in range(len(next_roll)):
                next_roll[j] = [i] + next_roll[j]
            output += next_roll
            
        return output
            
    return _rolls(amt, 1)

In [17]:
calc_cat = {
    'aces' : lambda x : x.count(1),
    'twos' : lambda x : x.count(2)*2,
    'threes' : lambda x : x.count(3)*3,
    'fours' : lambda x : x.count(4)*4,
    'fives' : lambda x : x.count(5)*5,
    'sixes' : lambda x : x.count(6)*6,
    
    '3_kind' : score_3ofkind,
    '4_kind' : score_4ofkind,
    'full_house': score_fullhouse,
    's_straight' : score_smsrtaight,
    'l_straight' : score_lstraight,
    'yahtzee' : score_yahtzee,
    'chance' : lambda x : sum(x)
}

In [18]:
scores = {
    'aces' : -1,
    'twos' : -1,
    'threes' : -1,
    'fours' : -1,
    'fives' : -1,
    'sixes' : -1,
    
    '3_kind' : -1,
    '4_kind' : -1,
    'full_house': -1,
    's_straight' : -1,
    'l_straight' : -1,
    'yahtzee' : -1,
    'chance' : -1,
}

In [19]:
def score_card_roll(roll, scorecard):
    scorecard = scorecard.copy()
    
    for key in scores.keys():
        if scorecard[key] == -1:
            scorecard[key] = calc_cat[key](roll)
            
    return scorecard

In [20]:
rolls_scorecards = {}

for roll in cmb_rolls(5):
    roll.sort()
    
    # key = roll as string
    key = ''.join(str(x) for x in roll)
    
    s = score_card_roll(roll, scores)
    
    rolls_scorecards[key] = s

<ipython-input-10-c7caafff7242>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "score_3ofkind" failed type inference due to: Untyped global name 'sum': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-10-c7caafff7242>", line 5:
def score_3ofkind(roll):
    <source elided>
        if roll.count(e) >= 3:
            return sum(roll)
            ^

  @jit()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "score_3ofkind" was compiled in object mode without forceobj=True.

File "<ipython-input-10-c7caafff7242>", line 2:
@jit()
def score_3ofkind(roll):
^

  state.func_ir.loc))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numba/object_mode_passes.py:187: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, th

In [21]:
counter = {}
rolls = cmb_rolls(5)
fraction = Fraction(1, len(rolls))
for roll in rolls:
    roll.sort()
    key = ''.join(str(x) for x in roll)
    if key in counter.keys():
        counter[key] += fraction
    else:
        counter[key] = fraction



In [22]:
def get_roll_scorecard(roll):
    roll.sort()
    return rolls_scorecards[''.join(str(x) for x in roll)]

In [23]:
def get_final_roll_value(scorecard):
    rolls_scorecards = {}

    for roll in cmb_rolls(5):
        roll.sort()

        # key = roll as string
        key = ''.join(str(x) for x in roll)

        s = score_card_roll(roll, scores)

        rolls_scorecards[key] = s

In [24]:
def upper_total(score_card):
    total = 0
    for cat in ['aces', 'twos', 'threes', 'fours', 'fives', 'sixes']:
        total += score_card[cat] if score_card[cat] != -1 else 0
        
    return total

In [25]:
def max_ev_roll(scorecard, roll):
    card = get_roll_scorecard(roll)
    
    upper_section = upper_total(score_card)
    upper_section_categories = ['aces', 'twos', 'threes', 'fours', 'fives', 'sixes']
    
    best_key = ''
    best_ev = -1
    
    for key in scorecard.keys():
        if scorecard[key] == -1:
            ev = card[key]
            
            if key in upper_section_categories:
                if upper_section < 63 and upper_section + ev >= 63:
                    ev += 35
            
            if ev > best_ev:
                best_key = key
                best_ev = ev
            
    return best_ev, best_key

In [26]:
# berechnet die keeper eines gegebenen rolls, welche beim aktuellen schritt bei einem reroll 
# im erwartungswert den höchsten wert haben.
# die funktion müsste sich rerkuriv aufrufen lassen, um so die keeper für 2 rerolls zu berechnen.
# durch rekursive aufrufe mit kopien der scorecard, in welchen die kategorie der jeweiligen
# 'besten' ev rerolls ausgefüllt sind müsste sich die langfristig für den erwartungswert 
# beste strategie berechnen lasseb müssen

def max_ev_keeper(scorecard, roll):
    keepers = cmb_keepers(roll)
    
    cache = {}
    
    max_ev = 0
    best_keepers = roll.copy()
    
    for keeper in keepers:
        keeper = list(keeper)
        
        keeper_cnt = len(keeper)
        if keeper_cnt < 5:
            
            ev = 0
            rerolls = cmb_rolls(5-keeper_cnt)
            for reroll in rerolls:
                reroll += keeper
                reroll.sort()

                key = ''.join(str(x) for x in reroll)

                if key in cache.keys():
                    ev += counter[key] * cache[key]
                else:
                    reroll_ev = max_ev_roll(scorecard, reroll)[0]
                    cache[key] = reroll_ev
                    ev += counter[key] * reroll_ev

            ev /= len(rerolls)
            
        else:
            keeper.sort()
            key = ''.join(str(x) for x in keeper)
            if key in cache.keys():
                ev = counter[key] * cache[key]
            else:
                reroll_ev = max_ev_roll(scorecard, keeper)[0]
                cache[key] = reroll_ev
                ev = counter[key] * reroll_ev
        
        if ev > max_ev:
            max_ev = ev
            best_keepers = keeper
                
                
    return best_keepers, ev

In [27]:
# naive implementation für 2 rerolls, bedarf evtl optimierung

def max_ev_keeper2(scorecard, roll):
    keepers = cmb_keepers(roll)
    
    cache = {}
    
    max_ev = 0
    best_keepers = roll.copy()
    
    
    for keeper in keepers:
        keeper = list(keeper)
        
        keeper_cnt = len(keeper)
        if keeper_cnt < 5:
            ev = 0
            
            rerolls = cmb_rolls(5-keeper_cnt)
            for reroll in rerolls:
                reroll += keeper
                reroll.sort()

                key = ''.join(str(x) for x in reroll)

                if key in cache.keys():
                    ev += counter[key] * cache[key]
                else:
                    reroll_ev = max_ev_keeper(scorecard, reroll)[1]
                    cache[key] = reroll_ev
                    ev += counter[key] * reroll_ev

            ev /= len(rerolls)
            
        else:
            keeper.sort()
            key = ''.join(str(x) for x in keeper)
            if key in cache.keys():
                ev = counter[key] * cache[key]
            else:
                reroll_ev = max_ev_keeper(scorecard, keeper)[1]
                cache[key] = reroll_ev
                ev = counter[key] * reroll_ev
        
        if ev > max_ev:
            max_ev = ev
            best_keepers = keeper
            
    return best_keepers, ev

In [28]:
score_card = {
    'aces' : -1,
    'twos' : -1,
    'threes' : -1,
    'fours' : -1,
    'fives' : -1,
    'sixes' : -1,
    
    '3_kind' : -1,
    '4_kind' : -1,
    'full_house': -1,
    's_straight' : -1,
    'l_straight' : -1,
    'yahtzee' : -1,
    'chance' : -1,
}

In [29]:
# beispielhafte greedy strategie
roll = roll_dices()
roll

[1, 6, 5, 4, 2]

In [30]:
keeper, expected_value = max_ev_keeper2(score_card, roll)
keeper

[6, 5, 4]

In [31]:
roll = roll_dices(keepers=keeper)
roll

[6, 5, 4, 6, 3]

In [32]:
keeper, expected_value = max_ev_keeper(score_card, roll)
keeper

[6, 5, 4, 3]

In [33]:
roll = roll_dices(keepers=keeper)
roll

[6, 5, 4, 3, 4]

In [34]:
max_ev_roll(score_card, roll)

(30, 's_straight')